# Основы анализа данных в Python

*Алла Тамбовцева*

## Практикум 5. Визуализация данных: часть 2

Импортируем библиотеку `pandas` и модуль `pyplot` из графической библиотеки `matplotlib`:

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

В файле Excel хранятся результаты выборов в Государственную Думу 2021 года. Загрузим данные с третьего листа файла Excel (`sheet_name = 2`).

In [ ]:
res = pd.read_excel("proportional_w_uiks.xlsx", sheet_name = 2)

### Сюжет 1: подготовка данных

* Удаление и переименование столбцов
* Добавление новых столбцов
* Группировка и агрегирование
* Вычисление максимума в каждой строке

Посмотрим на названия столбцов в `res`:

In [ ]:
print(res.columns)

Так как некоторые столбцы нам не понадобятся, можем их удалить, воспользовавшись методом `.drop()`. В исходном датафрейме `res` их оставим, а результат после удаления сохраним в новый датафрейм `small`:

In [ ]:
small = res.drop(columns = ["level", "oik", "uiknum", 
                           "Число утраченных избирательных бюллетеней",
                           "Число избирательных бюллетеней, не учтенных при получении"])
small.head()

Переименуем оставшиеся столбцы, так как названия у них слишком длинные. Просто сохраним в атрибут `.columns` список новых названий:

In [ ]:
small.columns = ["region", "district", "tik", "uik", 
                 "invalid", "valid", "КПРФ", "ЗЕЛЁНЫЕ", 
                 "ЛДПР", "НЛ", "ЕР", "СР", "ЯБЛОКО", 
                 "ПАРТИЯ РОСТА", "РПСС", "КП", "ГП", 
                 "ЗЕЛЁНАЯ АЛЬТЕРНАТИВА", "РОДИНА", 
                 "ПП"]

Добавим столбец `turnout` с явкой в абсолютных значениях (число бюллетеней, не проценты):

In [ ]:
small["turnout"] = small["invalid"] + small["valid"]
small.head()

Посмотрим на размерность датафрейма `small`:

In [ ]:
print(small.shape)

В датафрейме хранятся данные в разрезе избирательных участков, одна строка таблицы – один участок. Чтобы перейти к анализу результатов выборов по регионам, данные нужно агрегировать. Каким образом? Так как все числовые столбцы здесь в одинаковых единицах измерения – количество бюллетеней – для получения данных по регионам, строки нужно сгруппировать по региону и для каждого региона посчитать сумму по каждому столбцу:

In [ ]:
# в новых версия pandas аргумент numeric_only=True обязателен,
# чтобы игнорировались нечисловые столбцы,
# если у вас более старая версия (выдается ошибка), 
# оставьте просто .sum()

regs = small.groupby("region").sum(numeric_only=True)
regs.head()

Проверяем размерность:

In [ ]:
regs.shape # все ок

Как перейти от абсолютных чисел к процентам? Процент голосов, полученных на выборах партией, вычисляется от явки. Значит, нам нужно написать функцию, которая будет делить каждый столбец с результами партии на явку и домножать ответ на 100. Однако сначала выберем подходящие столбцы, столбцы с голосами за разные партии. Для этого, чтобы не писать длинный список, можно воспользоваться методами `.iloc[]` или `.loc[]`:

* метод `.iloc[]` выбирает столбцы по индексам;
* метод `.loc[]` выбирает столбцы по названиям.

На первом месте в квадратных скобках указываются индексы/названия выбираемых строк, на втором – выбираемых столбцов. Если мы выбираем все строки/все столбцы, можем указать полный срез `:`:

In [ ]:
# все строки, первый столбец с индексом 0
regs.iloc[:, 0]

In [ ]:
# все столбцы, вторая строка с индексом 1
regs.iloc[1, :]

Выберите, используя

* метод `.loc[]`
* метод `.iloc[]`

столбцы, которые соответствуют количеству голосов за каждую партию и сохраните в датафрейм `parties`.

In [ ]:
### YOUR CODE HERE ###

Напишите lambda-функцию, которая вычисляет процент голосов от явки и примените ее ко всем столбцам в `parties`, используя метод `.apply()`. Полученный датафрейм с процентами голосов назовите `regs_perc`.

In [ ]:
### YOUR CODE HERE ###

In [ ]:
regs_perc = regs.iloc[:, 2:-1].apply(lambda x: x / regs["turnout"] * 100)

Добавим столбец `winner` с названием партии, которая получила большинство голосов в регионе. Чтобы это сделать, нужно определить максимум по каждой строке. И даже не сам максимум, а название столбца, в котором этот максимум по строке достигается. Для этого в `pandas` есть универсальный метод `.idxmax()`:

In [ ]:
# axis = 1, так как ищем название столбца (0 – строка, 1 – столбец)

regs_perc["winner"] = regs_perc.idxmax(axis=1)

### Сюжет 2: построение круговой и кольцевой диаграммы

Выведите таблицу частот для нового столбца `winner` и сохраните ее в переменную `tab`:

In [ ]:
### YOUR CODE HERE ###

Постройте на основе полученной таблицы круговую диаграмму – та же логика, что и для столбиковой, только функция `pie()` вместо `bar()`.

In [ ]:
# ради всего святого, не стройте базовые круговые диаграммы, это катастрофа,
# но кольцевую без круговой не построишь

### YOUR CODE HERE ###

Доработаем график! Сначала разберем таблицу частот на части, так будет удобнее:

In [ ]:
labs = tab.index
vals = tab.values

print(labs)
print(vals)

Создадим заготовку для графика, пользуясь возможностями `matplotlib`:

In [ ]:
# fig – картинка, ее потом выгрузим в файл
# ax – оси, в которые вписываем элементы графика и редактируем
# figsize – размер изображения в дюймах
# dpi – разрешение, в точках на дюйм

fig, ax = plt.subplots(figsize = (16, 9), dpi = 300)

К осям `ax` применим метод `.pie()`, на первом месте укажем частоты, на втором – названия категорий:

In [ ]:
fig, ax = plt.subplots(figsize = (16, 9), dpi = 300)
ax.pie(vals, labels = labs)

Добавим подписи с процентами:

In [ ]:
fig, ax = plt.subplots(figsize = (16, 9), dpi = 300)

# autopct: автоподсчет процентов и их добавление в заданном формате
# pctdistance: расстояние подписей с процентами от центра
# textprops: словарь с настройками для текста

ax.pie(vals, labels = labs, 
       autopct = '%.1f%%', 
       pctdistance = 0.7,
       textprops = dict(fontsize = 14));

Даже в таком виде диаграмма выглядит не очень. Перейдём на кольцевую диаграмму (*ring diagram* или *donut diagram*). Python не умеет строить такую диаграмму сразу, поэтому нужно построить круговую диаграмму, а затем наложить на неё белый круг подходящего радиуса:

In [ ]:
fig, ax = plt.subplots(figsize = (16, 9), dpi = 300)

# остается, как прежде, увеличили pctdistance
ax.pie(vals, labels = labs, 
       autopct = '%.1f%%', 
       pctdistance = 0.8,
       textprops = dict(fontsize = 14));

# создаем белый круг радиуса 0.6 с центром в нуле
centre_circle = plt.Circle((0, 0), 0.6, fc = 'white')

# добавляем круг – add_artist()
# в текущие оси графика – gca() от get current axes
fig.gca().add_artist(centre_circle);

Обычно у круговых и кольцевых диаграмм названия категорий выносят в легенду, чтобы они не «болтались» вокруг круга. Для этого нужно «разобрать» диаграмму, создаваемую через `.pie()`, на части. Функция `.pie()` возвращает три объекта:

* набор секторов на круге (*patches* или *wedges*);
* набор подписей к секторам диаграммы (*labels*);
* набор подписей с процентами (если есть аргумент `autopct`).

In [ ]:
# посмотрим, как это выглядит
# wedges – сектора, они же patches
# texts – подписи, они же labels
# autotexts – подписи с процентами

wedges, texts, autotexts = ax.pie(vals, labels = labs, 
                                  autopct = '%.1f%%', 
                                  pctdistance = 0.8,
                                  textprops = dict(fontsize = 14));

print(wedges)
print(texts)
print(autotexts)

Теперь, чтобы убрать подписи `КПРФ` и `ЕР`, нужно убрать аргумент `labels`, а подписи из `labs` перенести в метод `.legend()` для добавления легенды:

In [ ]:
fig, ax = plt.subplots(figsize = (16, 9), dpi = 300)

# нет labels

wedges, texts, autotexts = ax.pie(vals, 
                                  autopct = '%.1f%%',
                                  pctdistance = 0.8, 
                                  textprops = dict(fontsize = 16));
# выносим названия в легенду
# легенда с заливкой и подписями
# фиксируем верний левый угол блока с легендой

ax.legend(wedges, labs,
          title = "Партия",
          loc = "upper left")

# все тот же код для белого круга
centre_circle = plt.Circle((0, 0), 0.6, fc = 'white')
fig.gca().add_artist(centre_circle);

In [ ]:
# добавляем свои цвета, исправляем легенду
# палитры https://coolors.co/

cols = ["#457b9d", "#e63946"]

fig, ax = plt.subplots(figsize = (16, 9), dpi = 300)

wedges, texts, autotexts = ax.pie(vals, 
                                  colors = cols,
                                  autopct = "%.1f%%",
                                  pctdistance = 0.8,
                                  textprops = dict(fontsize = 14));

# выключаем рамку у легенды
ax.legend(wedges, labs,
          loc = "upper left",
          frameon = False);

# добавляем заголовок графика
ax.set_title("Выборы в Государственную Думу 2021 года\nПобедившая партия (% регионов)", 
            fontsize = 16, 
            loc = "left", 
            fontweight = "bold");

# все тот же код для белого круга
centre_circle = plt.Circle((0, 0), 0.6, fc = "white")
fig.gca().add_artist(centre_circle);

В примерах выше мы везде применяли методы к осям графика `ax`. Однако на выходе нам нужны не сами оси, а материальная картинка, изображение, которое можно экспортировать в файл. А это – объект `fig`. Выберем его и сохраним в файл `my_pie.png`:

In [ ]:
fig.savefig("my_pie.png")